# 🤖 SENTALIS — Sentiment Analysis
## TF-IDF + Naive Bayes (Machine Learning Edition)

**Input:** Dataset berlabel dari SENTALIS Rule-Based (`dataset_ml.csv`)  
**Output:** Model terlatih + laporan evaluasi lengkap

---

### 🗺️ Alur Notebook
```
1. Load & eksplorasi dataset
2. Analisis distribusi kelas (imbalanced data)
3. Feature engineering dengan TF-IDF
4. Training Naive Bayes (Multinomial & Complement)
5. Evaluasi model — accuracy, precision, recall, F1
6. Confusion matrix & visualisasi
7. Analisis kesalahan prediksi
8. Prediksi teks baru
9. Simpan model
```

> 📁 Pastikan file `dataset_ml.csv` atau `dataset_train.csv` / `dataset_test.csv`  
> tersedia di direktori yang sama, atau upload ke Google Colab.

## 📦 Cell 1 — Install & Import

In [ ]:
!pip install scikit-learn pandas matplotlib seaborn imbalanced-learn joblib -q

import pandas as pd
import numpy as np
import re
import json
import joblib
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

# Scikit-learn: preprocessing & model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import (
    accuracy_score, classification_report,
    confusion_matrix, ConfusionMatrixDisplay,
    f1_score, precision_score, recall_score
)
from sklearn.utils.class_weight import compute_class_weight

# Imbalanced-learn: tangani kelas tidak seimbang
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline as ImbPipeline

plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.family'] = 'DejaVu Sans'

WARNA = {'Negatif': '#FF3B30', 'Netral': '#8E8E93', 'Positif': '#34C759'}
LABEL_ORDER = ['Negatif', 'Netral', 'Positif']

print('✅ Semua library berhasil diimport!')

## 📂 Cell 2 — Load Dataset

In [ ]:
# ─── Prioritas: gunakan dataset_ml.csv (full), atau split jika sudah ada ───
import os

USE_PRESPLIT = os.path.exists('dataset_train.csv') and os.path.exists('dataset_test.csv')

if USE_PRESPLIT:
    df_train_raw = pd.read_csv('dataset_train.csv', encoding='utf-8-sig')
    df_test_raw  = pd.read_csv('dataset_test.csv',  encoding='utf-8-sig')
    print(f'✅ Menggunakan pre-split dataset')
    print(f'   Train: {len(df_train_raw)} | Test: {len(df_test_raw)}')
else:
    # Load full dataset dan split manual
    for enc in ['utf-8-sig', 'utf-8', 'latin-1']:
        try:
            df_full = pd.read_csv('dataset_ml.csv', encoding=enc)
            print(f'✅ Loaded dataset_ml.csv ({enc})')
            break
        except Exception:
            continue
    USE_PRESPLIT = False

# Fungsi normalisasi dataframe
def normalize_df(df):
    df = df.copy()
    df['teks_bersih'] = df['teks_bersih'].fillna('').astype(str).str.strip()
    df = df[df['teks_bersih'].str.len() > 0].reset_index(drop=True)
    return df

if USE_PRESPLIT:
    df_train_raw = normalize_df(df_train_raw)
    df_test_raw  = normalize_df(df_test_raw)
    df_full = pd.concat([df_train_raw, df_test_raw], ignore_index=True)
else:
    df_full = normalize_df(df_full)

print(f'\n📊 Total data: {len(df_full)}')
print(f'📋 Kolom    : {list(df_full.columns)}')
display(df_full[['teks_bersih', 'label', 'label_id']].head(6))

## 🔍 Cell 3 — Eksplorasi & Analisis Distribusi Kelas

In [ ]:
dist = df_full['label'].value_counts()
total = len(df_full)

print('=' * 50)
print('📊 DISTRIBUSI KELAS')
print('=' * 50)
for label in LABEL_ORDER:
    n = dist.get(label, 0)
    pct = n / total * 100
    bar = '█' * int(pct / 2.5)
    print(f'  {label:10s} {bar:30s} {n:4d} ({pct:.1f}%)')
print(f'  Total     {" " * 30} {total}')

# Hitung imbalance ratio
n_max = dist.max()
n_min = dist.min()
ratio = n_max / n_min
print(f'\n⚠️  Imbalance Ratio: {ratio:.1f}x')
print(f'   Kelas mayoritas ({dist.idxmax()}): {n_max}')
print(f'   Kelas minoritas ({dist.idxmin()}): {n_min}')

if ratio > 5:
    print('\n❗ Dataset SANGAT TIDAK SEIMBANG.')
    print('   Akan digunakan strategi: class_weight + ComplementNB + oversampling')
elif ratio > 2:
    print('\n⚡ Dataset cukup tidak seimbang, akan digunakan class_weight.')
else:
    print('\n✅ Dataset cukup seimbang.')

# Statistik panjang teks per kelas
print('\n─── Rata-rata panjang teks per kelas ───')
df_full['n_kata'] = df_full['teks_bersih'].str.split().str.len()
for label in LABEL_ORDER:
    sub = df_full[df_full['label'] == label]['n_kata']
    if len(sub) > 0:
        print(f'  {label:10s}: rata {sub.mean():.1f} kata, max {sub.max()} kata')

# Visualisasi
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
fig.patch.set_facecolor('#F2F2F7')

# Bar distribusi
ax = axes[0]
ax.set_facecolor('white')
bars = ax.bar(LABEL_ORDER,
              [dist.get(l, 0) for l in LABEL_ORDER],
              color=[WARNA[l] for l in LABEL_ORDER],
              edgecolor='white', width=0.55)
for bar, label in zip(bars, LABEL_ORDER):
    h = bar.get_height()
    pct = h / total * 100
    ax.text(bar.get_x() + bar.get_width() / 2, h + 2,
            f'{int(h)}\n({pct:.1f}%)', ha='center', fontsize=10, fontweight='bold')
ax.set_title('Distribusi Kelas', fontweight='bold', fontsize=13)
ax.set_ylabel('Jumlah Sampel')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Boxplot panjang teks
ax2 = axes[1]
ax2.set_facecolor('white')
data_box = [df_full[df_full['label'] == l]['n_kata'].values for l in LABEL_ORDER]
bp = ax2.boxplot(data_box, patch_artist=True, labels=LABEL_ORDER,
                 medianprops=dict(color='white', linewidth=2))
for patch, label in zip(bp['boxes'], LABEL_ORDER):
    patch.set_facecolor(WARNA[label])
    patch.set_alpha(0.8)
ax2.set_title('Sebaran Panjang Teks (kata)', fontweight='bold', fontsize=13)
ax2.set_ylabel('Jumlah Kata')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

plt.tight_layout(pad=2)
plt.savefig('ml_distribusi_kelas.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()

## ✂️ Cell 4 — Train / Test Split

In [ ]:
X = df_full['teks_bersih'].values
y = df_full['label'].values

if USE_PRESPLIT:
    X_train = df_train_raw['teks_bersih'].values
    y_train = df_train_raw['label'].values
    X_test  = df_test_raw['teks_bersih'].values
    y_test  = df_test_raw['label'].values
    print('✅ Menggunakan pre-split (dari file train/test terpisah)')
else:
    # Stratified split — proporsional tiap kelas
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.2,
        random_state=42,
        stratify=y
    )
    print('✅ Stratified split 80/20 selesai')

print(f'\n   Train : {len(X_train)} sampel')
print(f'   Test  : {len(X_test)} sampel')

# Verifikasi distribusi di train & test
print('\n─── Distribusi label di setiap split ───')
for split_name, y_split in [('Train', y_train), ('Test', y_test)]:
    vals, counts = np.unique(y_split, return_counts=True)
    d = dict(zip(vals, counts))
    print(f'  {split_name}: ', end='')
    print(' | '.join([f'{l}={d.get(l,0)}' for l in LABEL_ORDER]))

## 🔢 Cell 5 — TF-IDF Vectorizer

> **TF-IDF** (Term Frequency–Inverse Document Frequency) mengubah teks menjadi
> vektor angka. Kata yang sering muncul di satu dokumen tapi jarang di seluruh
> corpus akan mendapat skor tinggi — artinya kata itu *deskriptif*.

In [ ]:
# Stopwords gabungan Indonesia + Sunda
STOPWORDS = [
    'yang', 'dan', 'di', 'ke', 'dari', 'ini', 'itu', 'ada', 'dengan',
    'untuk', 'pada', 'akan', 'adalah', 'juga', 'bisa', 'nya', 'saja',
    'sudah', 'agar', 'atau', 'karena', 'jika', 'bila',
    'kita', 'kami', 'saya', 'kamu', 'dia', 'mereka', 'anda',
    'bapak', 'ibu', 'pak', 'bu', 'pa', 'min', 'kak', 'bang',
    # Sunda
    'mah', 'teh', 'wae', 'oge', 'eta', 'atuh', 'yeuh', 'euy',
    'naha', 'kumaha', 'naon', 'iraha', 'keneh',
    # Chat slang
    'banget', 'bgt', 'aja', 'ajah', 'dong', 'deh', 'yah', 'ya',
    'yg', 'jd', 'tp', 'dgn', 'utk', 'krn', 'pd', 'sm', 'dr',
]

# Konfigurasi TF-IDF
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),        # unigram + bigram
    min_df=2,                  # abaikan kata yang muncul < 2 kali
    max_df=0.90,               # abaikan kata yang terlalu umum (>90% dokumen)
    max_features=3000,         # maksimal 3000 fitur
    sublinear_tf=True,         # gunakan log(TF) agar tidak terlalu dominan
    strip_accents='unicode',
    analyzer='word',
    stop_words=STOPWORDS
)

# Fit pada data train, transform train & test
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf  = tfidf.transform(X_test)

print('✅ TF-IDF Vectorizer selesai')
print(f'   Jumlah fitur (vocabulary): {len(tfidf.vocabulary_)}')
print(f'   Dimensi matrix train     : {X_train_tfidf.shape}')
print(f'   Dimensi matrix test      : {X_test_tfidf.shape}')

# Tampilkan top fitur global
feature_names = tfidf.get_feature_names_out()
mean_tfidf = np.asarray(X_train_tfidf.mean(axis=0)).flatten()
top_idx = mean_tfidf.argsort()[::-1][:20]

print('\n─── Top 20 fitur TF-IDF (rata-rata tertinggi) ───')
print(' | '.join(feature_names[top_idx]))

## 🧠 Cell 6 — Training Model Naive Bayes

Kita latih **dua varian** Naive Bayes:

| Model | Kelebihan |
|---|---|
| **MultinomialNB** | Standar untuk klasifikasi teks, cepat |
| **ComplementNB** | Dirancang khusus untuk data tidak seimbang (*imbalanced*) |

In [ ]:
# Hitung class weight untuk tangani imbalanced data
classes = np.unique(y_train)
weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, weights))

print('⚖️  Class weights (untuk tangani imbalanced data):')
for label, w in sorted(class_weight_dict.items()):
    print(f'   {label:10s}: {w:.3f}')
print()

# ─── Model 1: MultinomialNB ───
mnb = MultinomialNB(alpha=0.5)   # alpha = smoothing Laplace
mnb.fit(X_train_tfidf, y_train)
y_pred_mnb = mnb.predict(X_test_tfidf)
acc_mnb = accuracy_score(y_test, y_pred_mnb)
f1_mnb  = f1_score(y_test, y_pred_mnb, average='macro', zero_division=0)

# ─── Model 2: ComplementNB (lebih baik untuk imbalanced) ───
cnb = ComplementNB(alpha=0.5)
cnb.fit(X_train_tfidf, y_train)
y_pred_cnb = cnb.predict(X_test_tfidf)
acc_cnb = accuracy_score(y_test, y_pred_cnb)
f1_cnb  = f1_score(y_test, y_pred_cnb, average='macro', zero_division=0)

print('📈 Hasil awal (sebelum tuning):')
print(f'   MultinomialNB  — Accuracy: {acc_mnb:.4f} | Macro F1: {f1_mnb:.4f}')
print(f'   ComplementNB   — Accuracy: {acc_cnb:.4f} | Macro F1: {f1_cnb:.4f}')

# Pilih model terbaik
if f1_cnb >= f1_mnb:
    best_model = cnb
    best_name  = 'ComplementNB'
    y_pred_best = y_pred_cnb
else:
    best_model = mnb
    best_name  = 'MultinomialNB'
    y_pred_best = y_pred_mnb

print(f'\n🏆 Model terpilih: {best_name}')

## 🔁 Cell 7 — Cross Validation (5-Fold Stratified)

In [ ]:
# Pipeline lengkap: TF-IDF + model
pipeline_mnb = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1, 2), min_df=2, max_df=0.90,
        max_features=3000, sublinear_tf=True,
        stop_words=STOPWORDS
    )),
    ('clf', MultinomialNB(alpha=0.5))
])

pipeline_cnb = Pipeline([
    ('tfidf', TfidfVectorizer(
        ngram_range=(1, 2), min_df=2, max_df=0.90,
        max_features=3000, sublinear_tf=True,
        stop_words=STOPWORDS
    )),
    ('clf', ComplementNB(alpha=0.5))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print('🔄 Running 5-Fold Stratified Cross Validation...\n')

results = {}
for name, pipeline in [('MultinomialNB', pipeline_mnb), ('ComplementNB', pipeline_cnb)]:
    scores_acc = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')
    scores_f1  = cross_val_score(pipeline, X, y, cv=cv, scoring='f1_macro')
    results[name] = {
        'acc_mean': scores_acc.mean(),
        'acc_std':  scores_acc.std(),
        'f1_mean':  scores_f1.mean(),
        'f1_std':   scores_f1.std(),
        'acc_folds': scores_acc,
        'f1_folds':  scores_f1,
    }
    print(f'  {name}:')
    print(f'    Accuracy : {scores_acc.mean():.4f} ± {scores_acc.std():.4f}')
    print(f'    Macro F1 : {scores_f1.mean():.4f} ± {scores_f1.std():.4f}')
    print(f'    Per fold : {[f"{s:.3f}" for s in scores_f1]}')
    print()

# Visualisasi CV scores
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
fig.patch.set_facecolor('#F2F2F7')

for idx, metric in enumerate(['acc_folds', 'f1_folds']):
    ax = axes[idx]
    ax.set_facecolor('white')
    title = 'Accuracy' if idx == 0 else 'Macro F1'
    colors_cv = ['#007AFF', '#5856D6']
    fold_labels = [f'Fold {i+1}' for i in range(5)]

    x_pos = np.arange(5)
    w = 0.35
    for i, (name, color) in enumerate(zip(results.keys(), colors_cv)):
        ax.bar(x_pos + i * w, results[name][metric], w,
               label=name, color=color, alpha=0.85, edgecolor='white')

    ax.set_xticks(x_pos + w / 2)
    ax.set_xticklabels(fold_labels)
    ax.set_title(f'CV {title} per Fold', fontweight='bold')
    ax.set_ylim(0, 1.1)
    ax.legend(fontsize=9)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.savefig('ml_crossval.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()

## 📋 Cell 8 — Evaluasi Detail & Classification Report

In [ ]:
print('=' * 60)
print(f'📊 CLASSIFICATION REPORT — {best_name}')
print('=' * 60)
print(classification_report(y_test, y_pred_best, target_names=LABEL_ORDER,
                             zero_division=0))

# Metrik per kelas
precision = precision_score(y_test, y_pred_best, labels=LABEL_ORDER,
                            average=None, zero_division=0)
recall    = recall_score(y_test, y_pred_best, labels=LABEL_ORDER,
                         average=None, zero_division=0)
f1        = f1_score(y_test, y_pred_best, labels=LABEL_ORDER,
                     average=None, zero_division=0)
support   = [np.sum(y_test == l) for l in LABEL_ORDER]

df_report = pd.DataFrame({
    'Kelas':     LABEL_ORDER,
    'Precision': [f'{p:.3f}' for p in precision],
    'Recall':    [f'{r:.3f}' for r in recall],
    'F1-Score':  [f'{f:.3f}' for f in f1],
    'Support':   support,
})

macro_f1 = f1_score(y_test, y_pred_best, average='macro', zero_division=0)
acc      = accuracy_score(y_test, y_pred_best)

print(f'\n📈 Ringkasan:')
print(f'   Accuracy       : {acc:.4f} ({acc*100:.2f}%)')
print(f'   Macro F1       : {macro_f1:.4f}')
print(f'   Weighted F1    : {f1_score(y_test, y_pred_best, average="weighted", zero_division=0):.4f}')

print()
display(df_report)

# Visualisasi metrik per kelas
fig, ax = plt.subplots(figsize=(10, 4))
fig.patch.set_facecolor('#F2F2F7')
ax.set_facecolor('white')

x = np.arange(len(LABEL_ORDER))
w = 0.25
metrics_vals = [precision, recall, f1]
metric_names = ['Precision', 'Recall', 'F1-Score']
m_colors     = ['#007AFF', '#5856D6', '#34C759']

for i, (vals, name, color) in enumerate(zip(metrics_vals, metric_names, m_colors)):
    bars = ax.bar(x + i * w, vals, w, label=name, color=color, alpha=0.85, edgecolor='white')
    for bar, val in zip(bars, vals):
        if val > 0:
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
                    f'{val:.2f}', ha='center', va='bottom', fontsize=8, fontweight='bold')

ax.set_xticks(x + w)
ax.set_xticklabels(LABEL_ORDER, fontsize=11)
ax.set_ylim(0, 1.15)
ax.set_ylabel('Score')
ax.set_title(f'Precision / Recall / F1 per Kelas — {best_name}',
             fontweight='bold', fontsize=13)
ax.legend(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.axhline(y=macro_f1, color='gray', linestyle='--', linewidth=1, alpha=0.6,
           label=f'Macro F1 = {macro_f1:.3f}')
ax.legend(fontsize=9)

plt.tight_layout()
plt.savefig('ml_metrik_kelas.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()

## 🟦 Cell 9 — Confusion Matrix

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.patch.set_facecolor('#F2F2F7')

for ax, y_pred, model_name in zip(
    axes,
    [y_pred_mnb, y_pred_cnb],
    ['MultinomialNB', 'ComplementNB']
):
    cm = confusion_matrix(y_test, y_pred, labels=LABEL_ORDER)
    cm_pct = cm.astype(float) / cm.sum(axis=1, keepdims=True) * 100

    # Buat label dengan angka + persentase
    annot = np.array(
        [[f'{cm[i,j]}\n({cm_pct[i,j]:.0f}%)' for j in range(len(LABEL_ORDER))]
         for i in range(len(LABEL_ORDER))]
    )

    sns.heatmap(
        cm_pct, annot=annot, fmt='',
        xticklabels=LABEL_ORDER, yticklabels=LABEL_ORDER,
        cmap='Blues', ax=ax,
        linewidths=0.5, linecolor='#F2F2F7',
        cbar_kws={'shrink': 0.8}
    )
    acc_m = accuracy_score(y_test, y_pred)
    ax.set_title(f'{model_name}\nAccuracy: {acc_m:.4f}', fontweight='bold', fontsize=12)
    ax.set_xlabel('Prediksi', fontsize=11)
    ax.set_ylabel('Aktual', fontsize=11)
    ax.set_facecolor('white')

plt.suptitle('Confusion Matrix (% dari aktual)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('ml_confusion_matrix.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()

# Interpretasi diagonal
print('\n📝 Interpretasi Confusion Matrix:')
cm_best = confusion_matrix(y_test, y_pred_best, labels=LABEL_ORDER)
for i, label in enumerate(LABEL_ORDER):
    row_sum = cm_best[i].sum()
    correct = cm_best[i, i]
    if row_sum > 0:
        print(f'   {label:10s}: {correct}/{row_sum} benar ({correct/row_sum*100:.1f}% recall)')

## 🔤 Cell 10 — Fitur Paling Informatif per Kelas

In [ ]:
# Ambil log probability tiap fitur per kelas dari ComplementNB
feature_names = tfidf.get_feature_names_out()
N_TOP = 15

fig, axes = plt.subplots(1, 3, figsize=(16, 5))
fig.patch.set_facecolor('#F2F2F7')

# Dapatkan urutan kelas dari model
model_classes = list(best_model.classes_)

for ax, label in zip(axes, LABEL_ORDER):
    if label not in model_classes:
        ax.text(0.5, 0.5, f'Kelas {label}\ntidak ada di train',
                ha='center', va='center', transform=ax.transAxes)
        ax.axis('off')
        continue

    idx_class = model_classes.index(label)
    log_probs = best_model.feature_log_prob_[idx_class]

    # Top N fitur dengan probabilitas tertinggi
    top_idx = log_probs.argsort()[::-1][:N_TOP]
    top_features = feature_names[top_idx]
    top_scores   = np.exp(log_probs[top_idx])  # konversi ke probabilitas

    color = WARNA[label]
    ax.set_facecolor('white')
    ax.barh(list(reversed(top_features)), list(reversed(top_scores)),
            color=color, alpha=0.85, edgecolor='white')
    ax.set_title(f'Top Fitur — {label}', fontweight='bold', fontsize=12, color=color)
    ax.set_xlabel('P(kata | kelas)', fontsize=9)
    ax.tick_params(axis='y', labelsize=8)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

plt.suptitle(f'Fitur Paling Informatif per Kelas — {best_name}',
             fontsize=13, fontweight='bold', y=1.01)
plt.tight_layout()
plt.savefig('ml_top_fitur.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()

## ❌ Cell 11 — Analisis Kesalahan Prediksi

In [ ]:
# Buat dataframe hasil prediksi
df_hasil = pd.DataFrame({
    'teks':          X_test,
    'label_aktual':  y_test,
    'pred_mnb':      y_pred_mnb,
    'pred_cnb':      y_pred_cnb,
    'pred_best':     y_pred_best,
})

# Probabilitas prediksi (confidence)
prob_best = best_model.predict_proba(X_test_tfidf)
df_hasil['confidence'] = prob_best.max(axis=1)

# Tandai kesalahan
df_hasil['benar'] = df_hasil['label_aktual'] == df_hasil['pred_best']
df_salah = df_hasil[~df_hasil['benar']].copy()

print(f'✅ Prediksi benar : {df_hasil["benar"].sum()} / {len(df_hasil)}')
print(f'❌ Prediksi salah : {(~df_hasil["benar"]).sum()} / {len(df_hasil)}')

if len(df_salah) > 0:
    print(f'\n─── Pola Kesalahan ───')
    pola = df_salah.groupby(['label_aktual', 'pred_best']).size().reset_index(name='count')
    pola = pola.sort_values('count', ascending=False)
    for _, row in pola.iterrows():
        print(f'   {row["label_aktual"]:10s} → diprediksi {row["pred_best"]:10s}: {row["count"]} kasus')

    print(f'\n─── 10 Kesalahan dengan Confidence Tertinggi ───')
    df_salah_sorted = df_salah.sort_values('confidence', ascending=False).head(10)
    for _, row in df_salah_sorted.iterrows():
        teks_preview = row['teks'][:65] + '...' if len(row['teks']) > 65 else row['teks']
        print(f'   Aktual: {row["label_aktual"]:10s} | Prediksi: {row["pred_best"]:10s} | '
              f'Conf: {row["confidence"]:.3f}')
        print(f'   Teks  : "{teks_preview}"')
        print()

# Export hasil prediksi lengkap
df_hasil.to_csv('hasil_prediksi_test.csv', index=False, encoding='utf-8-sig')
print('💾 Tersimpan: hasil_prediksi_test.csv')

## 🧪 Cell 12 — Prediksi Teks Baru

In [ ]:
import re

def preprocess(text):
    if not isinstance(text, str): return ''
    text = text.lower().strip()
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^\w\s.,!?\'-]', ' ', text)
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def prediksi(teks_input, verbose=True):
    """
    Prediksi sentimen teks baru menggunakan model terlatih.
    Returns: dict dengan label, confidence, dan probabilitas tiap kelas
    """
    bersih = preprocess(teks_input)
    if not bersih:
        return {'label': 'Netral', 'confidence': 0.0, 'teks_bersih': ''}

    vec = tfidf.transform([bersih])
    label = best_model.predict(vec)[0]
    probs = best_model.predict_proba(vec)[0]
    prob_dict = dict(zip(best_model.classes_, probs))
    confidence = probs.max()

    if verbose:
        icon = {'Negatif': '🔴', 'Netral': '⚪', 'Positif': '🟢'}.get(label, '⚫')
        print(f'  Input     : "{teks_input[:70]}"')
        print(f'  Bersih    : "{bersih[:70]}"')
        print(f'  Prediksi  : {icon} {label}  (confidence: {confidence:.3f})')
        for l in LABEL_ORDER:
            p = prob_dict.get(l, 0)
            bar = '▓' * int(p * 20)
            print(f'  {l:10s}: {bar:20s} {p:.3f}')
        print()

    return {'label': label, 'confidence': confidence,
            'probabilitas': prob_dict, 'teks_bersih': bersih}


# ─── Uji coba prediksi ───
teks_uji = [
    'Alhamdulillah jalan depan rumah akhirnya diperbaiki, terima kasih pak bupati!',
    'Jalan masih rusak parah, kapan diperbaiki? Sudah bertahun-tahun begini.',
    'Cukup bagus kepemimpinan bapa bupati, semoga terus maju.',
    'Ga ngaruh apa2. Sama aja bohong. Hahaha.',
    'Hmm biasa aja sih, gak ada yang spesial.',
    'Pengangguran masih tinggi, lapangan kerja kurang, galian pasir masih beroperasi.',
]

print('=' * 60)
print('🧪 PREDIKSI TEKS BARU')
print('=' * 60)
for teks in teks_uji:
    prediksi(teks)

## 💾 Cell 13 — Simpan Model

In [ ]:
import zipfile

# Simpan model & vectorizer
joblib.dump(best_model, 'model_naive_bayes.pkl')
joblib.dump(tfidf, 'vectorizer_tfidf.pkl')
print(f'✅ Model tersimpan  : model_naive_bayes.pkl')
print(f'✅ Vectorizer simpan: vectorizer_tfidf.pkl')

# Simpan metadata model
model_info = {
    'nama_model':    best_name,
    'vectorizer':    'TF-IDF',
    'tfidf_params': {
        'ngram_range': '(1, 2)',
        'min_df': 2,
        'max_df': 0.90,
        'max_features': 3000,
        'sublinear_tf': True,
    },
    'hasil_evaluasi': {
        'accuracy':   round(float(accuracy_score(y_test, y_pred_best)), 4),
        'macro_f1':   round(float(f1_score(y_test, y_pred_best, average='macro', zero_division=0)), 4),
        'per_kelas': {
            label: {
                'precision': round(float(precision_score(y_test, y_pred_best, labels=[label], average='macro', zero_division=0)), 4),
                'recall':    round(float(recall_score(y_test, y_pred_best, labels=[label], average='macro', zero_division=0)), 4),
                'f1':        round(float(f1_score(y_test, y_pred_best, labels=[label], average='macro', zero_division=0)), 4),
                'support':   int(np.sum(y_test == label)),
            } for label in LABEL_ORDER
        },
    },
    'cross_validation': {
        model: {
            'accuracy_mean': round(results[model]['acc_mean'], 4),
            'f1_mean':       round(results[model]['f1_mean'], 4),
        } for model in results
    },
    'label_map':      {'Negatif': 0, 'Netral': 1, 'Positif': 2},
    'cara_pakai': [
        "import joblib",
        "model = joblib.load('model_naive_bayes.pkl')",
        "tfidf = joblib.load('vectorizer_tfidf.pkl')",
        "teks_bersih = preprocess(teks_baru)  # gunakan fungsi preprocess()",
        "X = tfidf.transform([teks_bersih])",
        "label = model.predict(X)[0]",
    ]
}

with open('model_info.json', 'w', encoding='utf-8') as f:
    json.dump(model_info, f, ensure_ascii=False, indent=2)
print('✅ Tersimpan: model_info.json')

# ZIP semua output
output_files = [
    'model_naive_bayes.pkl', 'vectorizer_tfidf.pkl', 'model_info.json',
    'hasil_prediksi_test.csv',
    'ml_distribusi_kelas.png', 'ml_crossval.png',
    'ml_metrik_kelas.png', 'ml_confusion_matrix.png', 'ml_top_fitur.png',
]

with zipfile.ZipFile('SENTALIS_ML_output.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
    for fname in output_files:
        if os.path.exists(fname):
            zf.write(fname)
            print(f'  + {fname}')

print(f'\n📦 ZIP: SENTALIS_ML_output.zip ({os.path.getsize("SENTALIS_ML_output.zip")/1024:.1f} KB)')

# Auto-download di Colab
try:
    from google.colab import files
    files.download('SENTALIS_ML_output.zip')
    print('⬇️  Download dimulai...')
except ImportError:
    print('📂 File tersimpan di direktori saat ini.')

## 🔮 Cell 14 — Load Model & Prediksi Ulang (Opsional)

> Cell ini untuk membuktikan bahwa model yang disimpan bisa diload dan digunakan
> tanpa perlu training ulang.

In [ ]:
# Load model dari file
model_loaded  = joblib.load('model_naive_bayes.pkl')
tfidf_loaded  = joblib.load('vectorizer_tfidf.pkl')

print('✅ Model & vectorizer berhasil di-load dari file')
print(f'   Tipe model: {type(model_loaded).__name__}')
print(f'   Kelas model: {model_loaded.classes_}')
print()

# Coba prediksi menggunakan model yang di-load
def prediksi_dari_file(teks):
    bersih = preprocess(teks)
    vec = tfidf_loaded.transform([bersih])
    label = model_loaded.predict(vec)[0]
    conf  = model_loaded.predict_proba(vec)[0].max()
    icon  = {'Negatif': '🔴', 'Netral': '⚪', 'Positif': '🟢'}.get(label, '⚫')
    print(f'  {icon} {label} ({conf:.3f}) | "{teks[:60]}"')

print('🔮 Test prediksi dari model yang di-load:')
prediksi_dari_file('Jalan rusak parah, sudah lama tidak diperbaiki!')
prediksi_dari_file('Terima kasih, programnya sangat bermanfaat untuk warga.')
prediksi_dari_file('Biasa aja, tidak ada yang berubah.')

print('\n✅ Model siap digunakan!')

---

## 📊 Ringkasan & Catatan Penting

### Tentang Imbalanced Data

Dataset ini memiliki distribusi yang sangat tidak seimbang:
- **Netral ~79%** — kelas mayoritas
- **Negatif ~17%** — kelas sedang  
- **Positif ~4%** — kelas minoritas ⚠️

Akibatnya, **accuracy tinggi bisa menyesatkan** — model bisa saja selalu memprediksi "Netral" dan tetap dapat accuracy >79%. Gunakan **Macro F1** sebagai metrik utama.

### Cara Tingkatkan Performa

| Strategi | Cara |
|---|---|
| **Tambah data Positif** | Cari/tambahkan lebih banyak komentar positif berlabel |
| **Human annotation** | Review manual terutama komentar Positif |
| **Ganti model** | Coba IndoBERT untuk pemahaman konteks lebih baik |
| **Tuning alpha** | Eksperimen nilai alpha 0.1 – 1.0 pada Naive Bayes |

### Cara Pakai Model

```python
import joblib

model = joblib.load('model_naive_bayes.pkl')
tfidf = joblib.load('vectorizer_tfidf.pkl')

teks = "Jalan masih rusak dan gelap!"
X = tfidf.transform([teks.lower()])
label = model.predict(X)[0]      # 'Negatif'
probs = model.predict_proba(X)   # array probabilitas
```

---
*SENTALIS — Machine Learning Edition*  
*TF-IDF + Naive Bayes (Multinomial & Complement)*